In [36]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

DEFINE THE FOLLOWING PARAMETERS

In [43]:
#Paths
# Input folder containing CSV files
input_folder = 'C:/Users/quirosgu/Desktop/OK/based_on_stauntonia/' #/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/V1V2/'

#PF_data_file = 'C:/Users/quirosgu/Desktop/FULL_PF_graph_collection'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

In [47]:
df =pd.read_csv('C:/Users/quirosgu/Desktop/OK/based_on_stauntonia/Latifoliol.csv')
df

,feature,doc,rt,parent_mass,count_inter,count_loss,count_peak,total_peak,jaccard_similarity
0,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,2.790150,697.206909,115,15,100,115,1.000000
1,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,3.041367,697.206726,81,17,100,117,0.536424
2,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,3.041417,697.206024,71,16,100,116,0.443750
3,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,2.811350,697.206940,67,13,91,104,0.440789
4,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,2.790483,697.206604,69,19,100,119,0.418182
...,...,...,...,...,...,...,...,...,...
4916,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,3.858383,293.154755,13,98,100,198,0.043333
4917,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,2.137217,299.164001,13,98,100,198,0.043333
4918,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,3.665383,315.086334,13,98,100,198,0.043333
4919,https://enpkg.commons-lab.org/kg/lcms_feature_...,https://enpkg.commons-lab.org/kg/spec2vec_doc_...,1.626650,247.132553,13,98,100,198,0.043333


In [48]:
# List all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Loop through each CSV file
for file_name in csv_files:
    # Construct the full file path
    file_path = os.path.join(input_folder, file_name)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Extract 'MassIVE' code from 'feature' column
    df['MassIVE'] = df['feature'].str.extract(r'mzspec:(MSV\d+):')

    # Extract 'ms_filename' from 'feature' column
    ms_filename_match = df['feature'].str.extract(r'[^:]+:[^:]+:([^:]+)_features_ms2_(pos|neg).mgf:')
    if ms_filename_match is not None and len(ms_filename_match.columns) == 2:
        df['ms_filename'] = ms_filename_match[0]
    
    # Extract 'scan' from 'feature' column
    df['scan'] = df['feature'].str.extract(r'scan:(\d+)')
    
    # Perform any additional modifications to the DataFrame here
    
    # Save the modified DataFrame back to the same file
    df.to_csv(file_path, index=False)  # Set index=False to exclude the index column


In [49]:
# List all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each CSV file
for file_name in csv_files:
    # Construct the full file path
    file_path = os.path.join(input_folder, file_name)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Filter rows where 'jaccard_similarity' > 0.1
    filtered_df = df[df['jaccard_similarity'] > 0.25]
    
    # Extract unique 'feature_query' and 'ms_filename' pairs
    unique_pairs = filtered_df[['feature', 'ms_filename']].drop_duplicates()
    
    # Append the unique pairs to the list of DataFrames
    dfs.append(unique_pairs)

# Concatenate all DataFrames into a single combined DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Group by 'ms_filename' and count occurrences
count_series = combined_df.groupby('ms_filename').size()

# Convert the series to a DataFrame with 'ms_filename' and 'count' columns
count_df = count_series.reset_index(name='count')

In [50]:
df_PF =pd.read_csv('C:/Users/quirosgu/Desktop/FULL_PF_graph_collection.csv', low_memory='False')
df_PF.head()

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_8468\425851110.py:1: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_PF =pd.read_csv('C:/Users/quirosgu/Desktop/FULL_PF_graph_collection.csv', low_memory='False')


,extract,PF_code,wd_id,taxon,organ
0,VGF138_A02,V111819,http://www.wikidata.org/entity/Q163018,Paeonia suffruticosa,leaves
1,VGF138_A03,V111988,http://www.wikidata.org/entity/Q4984693,Buddleja officinalis,leaves
2,VGF138_A04,V112020,http://www.wikidata.org/entity/Q10928313,Ficus tikoua,multiple
3,VGF138_A05,V112033,http://www.wikidata.org/entity/Q1509784,Nicandra physalodes,fruits
4,VGF138_A06,V112042,http://www.wikidata.org/entity/Q38197224,Asemeia extraaxillaris,roots


In [51]:
df_final = pd.merge(count_df, df_PF, how= 'left', left_on='ms_filename', right_on='extract')
df_final

,ms_filename,count,extract,PF_code,wd_id,taxon,organ
0,VGF138_A07,3,VGF138_A07,V112053,http://www.wikidata.org/entity/Q6747995,Sophora davidii,green_stems
1,VGF141_A06,1,VGF141_A06,V113028,http://www.wikidata.org/entity/Q3029292,Dipterocarpus tuberculatus,leaves
2,VGF141_A09,11,VGF141_A09,V113055,http://www.wikidata.org/entity/Q15049903,Gnetum edule,woody_stems
3,VGF141_A10,1,VGF141_A10,V113065,http://www.wikidata.org/entity/Q13110312,Holoptelea integrifolia,leaves
4,VGF141_B06,1,VGF141_B06,V113029,http://www.wikidata.org/entity/Q3029292,Dipterocarpus tuberculatus,woody_stems
5,VGF141_B09,38,VGF141_B09,V113056,http://www.wikidata.org/entity/Q15049903,Gnetum edule,roots
6,VGF141_B10,2,VGF141_B10,V113067,http://www.wikidata.org/entity/Q13110312,Holoptelea integrifolia,roots
7,VGF141_C06,3,VGF141_C06,V113030,http://www.wikidata.org/entity/Q3029292,Dipterocarpus tuberculatus,roots
8,VGF141_C10,2,VGF141_C10,V113068,http://www.wikidata.org/entity/Q5485837,Hopea helferi,leaves
9,VGF141_D06,1,VGF141_D06,V113031,http://www.wikidata.org/entity/Q3029294,Dipterocarpus turbinatus,leaves


In [52]:
# Save the combined DataFrame to a CSV file
df_final.to_csv(f'{input_folder} combined_results_js25.csv', index=False)


In [56]:
#work with the chemical classes query

df_chem =pd.read_csv('C:/Users/quirosgu/Desktop/OK/querycc.csv')
df_chem = pd.merge(df, df_PF, how= 'left', left_on='extract', right_on='extract')
# Save the combined DataFrame to a CSV file
df_chem.to_csv(f'{input_folder} 2_chem_class_output.csv', index=False)
df_chem.head()

,sample,extract,ccount,PF_code,wd_id,taxon,organ
0,https://enpkg.commons-lab.org/kg/VGF154_H04,VGF154_H04,95,V114227,http://www.wikidata.org/entity/Q15558429,Ampelocissus arachnoidea,multiple
1,https://enpkg.commons-lab.org/kg/VGF156_G10,VGF156_G10,68,V114434,http://www.wikidata.org/entity/Q1329892,Shorea roxburghii,bark
2,https://enpkg.commons-lab.org/kg/VGF141_B09,VGF141_B09,60,V113056,http://www.wikidata.org/entity/Q15049903,Gnetum edule,roots
3,https://enpkg.commons-lab.org/kg/VGF142_D04,VGF142_D04,56,V113102,http://www.wikidata.org/entity/Q5485804,Hopea chinensis,roots
4,https://enpkg.commons-lab.org/kg/VGF143_G10,VGF143_G10,48,V113270,http://www.wikidata.org/entity/Q10865526,Stauntonia brunoniana,roots
